In [ ]:
import os
import json
import csv
import pandas as pd
from pandas import *
from matplotlib import *
%matplotlib inline
from matplotlib.pyplot import *
from scipy.stats import ttest_ind,ttest_ind_from_stats
from scipy.stats import *
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
def affiliation_parsing(x: dict) -> str:
    current = []
    for key in ['laboratory', 'institution']:
        if x['affiliation'].get(key):  
            current.append(x['affiliation'][key])
        else:
            current.append('')
    for key in ['addrLine', 'settlement', 'region', 'country', 'postCode']:
        if x['affiliation'].get('location'):
            if x['affiliation']['location'].get(key):
                current.append(x['affiliation']['location'][key])
        else:
            current.append('')
    return ', '.join(current)


def cite_parsing(x: list, key: str) -> list:
    cites = [i[key] if i else '' for i in x]
    output = []
    for i in cites:
        if i:
            output.append(','.join([j['ref_id'] if j['ref_id'] else '' for j in i]))
        else:
            output.append('')
    return '|'.join(output)


def extract_key(x: list, key:str) -> str:
    if x:
        return ['|'.join(i[key] if i[key] else '' for i in x)]
    return ''

extract_func = lambda x, func: ['|'.join(func(i) for i in x)]
format_authors = lambda x: f"{x['first']} {x['last']}"
format_full_authors = lambda x: f"{x['first']} {''.join(x['middle'])} {x['last']} {x['suffix']}"
format_abstract = lambda x: "{}\n {}".format(x['section'], x['text'])
all_keys = lambda x, key: '|'.join(i[key] for i in x.values())

In [ ]:
for path in ['biorxiv_medrxiv1']:
    json_files = [file for file in os.listdir(f'/CORD-19-research-challenge/{path}') if file.endswith('.json')]
    df_list = []

    for js in json_files:
        with open(os.path.join(f'/CORD-19-research-challenge/{path}', js)) as json_file:
            paper = json.load(json_file)
            print(path, js)
            paper_df = pd.DataFrame({
            'paper_id': paper['paper_id'],
            'title': paper['metadata']['title'],
            'authors': extract_func(paper['metadata']['authors'], format_authors),
            'bib_titles': all_keys(paper['bib_entries'], 'title'),
        })
        df_list.append(paper_df)
    temp_df = pd.concat(df_list)
    temp_df.to_csv(f'/CORD-19-research-challenge/{path}.csv', index=False)

In [ ]:
data=read_csv('biorxiv_medrxiv.csv')
x = (data['bib_titles'])
a = data['paper_id']

In [ ]:
final = []
final1 = ""
for i in range(0,len(x)):
    y = (data['bib_titles'][i])
    final1 = y.split('|')
    final.append(final1)

In [ ]:
finalresult = []
for i in range(0,len(x)):
    for j in range(0,len(final[i])):
        tp1 = (a[i],final[i][j])
        finalresult.append(tp1)

In [ ]:
with open('biorxiv_medrxiv_output.csv','w',encoding="utf-8",newline='') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['paper_id','bib_title'])
    for row in finalresult:
        csv_out.writerow(row)

In [ ]:
df = pd.read_csv('biorxiv_medrxiv.csv')
key= df['paper_id']
bib= df['bib_titles'].str.split('|')
key_pair=[]
for y in range(0,len(key)):
    for x in bib[y]:
            key_pair.append((key[y],x))

In [ ]:
print(key_pair)

In [ ]:
srcdata=read_csv('metadata.csv')
dstdata=read_csv('cleandata_biorxiv_medrxiv.csv')
a=srcdata['title']
b=dstdata['refined_title']
c = srcdata['sha']
d = dstdata['paper_id']
res = []
for i in range(0,len(a)):
    for j in range(0,len(b)):
        if not pd.isnull(c[i]):
            val = fuzz.token_sort_ratio(a[i],b[j])
            tup1 = (c[i],d[j],a[i],b[j],val)
            res.append(tup1)

In [ ]:
with open('final_biorxiv_medrxiv.csv','w',encoding="utf-8",newline='') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['sha','paper_id','paper_name_meta','paper_name_folder','ratio'])
    for row in res:
        csv_out.writerow(row)